In [ ]:
#Ça installe le logiciel qui va permettre à ton simulateur d’envoyer des messages.
!pip install paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
import time
import json
import random
import paho.mqtt.client as mqtt
from collections import deque
from IPython.display import display
import plotly.graph_objects as go

# MQTT
BROKER = "broker.hivemq.com"
PORT = 1883
DEVICE_ID = "device123"

TOPIC = f"devices/{DEVICE_ID}/telemetry"
CMD_TOPIC = f"devices/{DEVICE_ID}/cmd"

# Stocker les données
temps = deque(maxlen=20)
humidities = deque(maxlen=20)

# -----------  CALLBACK COMMANDES (DOWNLINK)  ----------------
def on_cmd(client, userdata, msg):
    cmd = json.loads(msg.payload.decode())
    print("\n⚙️ COMMAND RECEIVED :", cmd)

# -----------  CALLBACK TÉLÉMÉTRIE  ----------------
def on_message(client, userdata, msg):
    data = json.loads(msg.payload.decode())
    temps.append(data["temp"])
    humidities.append(data["humidity"])
    print("\n📥 TELEMETRY RECEIVED :", data)

# -----------  MQTT SETUP  ----------------
client = mqtt.Client(client_id=DEVICE_ID)

# Bind callback général pour télémétrie
client.on_message = on_message

client.connect(BROKER, PORT, 60)

# Subscriptions
client.subscribe(TOPIC)                    # Télémétrie
client.message_callback_add(CMD_TOPIC, on_cmd)
client.subscribe(CMD_TOPIC)                # Commandes

client.loop_start()

# -----------  PUBLICATION SIMULÉE  ----------------
for i in range(10):
    data = {
        "device_id": DEVICE_ID,
        "temp": round(20 + random.random()*10, 2),
        "humidity": round(30 + random.random()*40, 2),
        "ts": int(time.time())
    }

    payload = json.dumps(data)
    client.publish(TOPIC, payload)

    print("\n📤 PUBLISHED MESSAGE :", data)

    time.sleep(2)

print("\n⏳ Attente de quelques secondes avant d’afficher le graphique...")
time.sleep(3)

# -----------  DASHBOARD  ----------------
fig = go.Figure()
fig.add_trace(go.Scatter(y=list(temps), mode='lines+markers', name='Température (°C)'))
fig.add_trace(go.Scatter(y=list(humidities), mode='lines+markers', name='Humidité (%)'))

fig.update_layout(
    title='Température & Humidité (données reçues)',
    xaxis_title='Derniers messages',
    yaxis_title='Valeurs',
    yaxis=dict(range=[0, 100])
)

display(fig)

client.disconnect()
